In [70]:
from sklearn.linear_model import RidgeClassifier
from adapt.utils import make_classification_da
from adapt.feature_based import CORAL,DeepCORAL
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn import datasets, linear_model
from sklearn import svm
from sklearn.metrics import mean_squared_error, r2_score

In [71]:
def get_data(data_type,file_num):

    if(data_type=='train'):
        data=pd.read_csv(f"Dataset/IHDP_a/ihdp_npci_train_{file_num}.csv")
        #data=pd.read_csv(f"Data/ihdp_npci_train_{file_num}.csv",index_col=False)
    else:
        data = pd.read_csv(f"Dataset/IHDP_a/ihdp_npci_test_{file_num}.csv")
        #data = pd.read_csv(f"Data/ihdp_npci_test_{file_num}.csv",index_col=False)

    x_data=pd.concat([data.iloc[:,0], data.iloc[:, 1:30]], axis = 1)
 
   
    y_data=data.iloc[:, 1]
    
    return x_data,y_data

In [72]:
reg_0 = svm.SVR()
reg_1=svm.SVR()

In [73]:
def cal_pehe(y_1,y_0,test_x):
    #pehe...
    cate_pred=y_1-y_0
    #cate_pred=y1-y0
    cate_true=test_x.iloc[:,4]-test_x.iloc[:,3] #Hill's noiseless true values
    cate_err=np.mean( np.square( ( (cate_true) - (cate_pred) ) ) )
    return np.sqrt(cate_err)

def coral(xs,xt):
    model = CORAL(svm.SVR(), Xt=xt, random_state=0,verbose=0)
    
    trans=model.transform(xs)
    #trant=model.transform(xt)
    
    return trans
"""
def dcoral(xs,y,xt):
    model = DeepCORAL(lambda_=1., Xt=Xt, metrics=["acc"], random_state=0)
    model.fit(xs, y, epochs=100, verbose=0)
    trans=model.transform(xs)
    transt=model.transform(xt)
    return trans,transt
"""
    
def main_func(fil_num):
    x_data,y_data=get_data('train',fil_num)
    test_x,test_y=get_data('test',fil_num)
    x_train_sr=x_data[x_data['treatment']==1]
    y_train_sr=y_data[x_data['treatment']==1]
    x_train_tr=x_data[x_data['treatment']==0]
    y_train_tr=y_data[x_data['treatment']==0]
    
    #--------------
    trans=coral(x_train_sr.iloc[:,5:30],x_train_tr.iloc[:,5:30])
    #trans,transt=dcoral(x_train_sr.iloc[:,5:30],x_train_sr.iloc[:,0].to_numpy(),x_train_tr.iloc[:,5:30])
    
    reg_1.fit(trans, y_train_sr)
    y_1=reg_1.predict(test_x.iloc[:,5:30])
    reg_0.fit(x_train_tr.iloc[:,5:30], y_train_tr)
    y_0=reg_0.predict(test_x.iloc[:,5:30])
    
    pehe=cal_pehe(y_1,y_0,test_x)
    return pehe

In [74]:
pehe_list=[]
for i in range(1,101):
    pehe_list.append(main_func(i))

/home/ahmad/envs/RL/lib/python3.10/site-packages/sklearn/base.py:413: UserWarning: X has feature names, but SVR was fitted without feature names
  warnings.warn(
/home/ahmad/envs/RL/lib/python3.10/site-packages/sklearn/base.py:413: UserWarning: X has feature names, but SVR was fitted without feature names
  warnings.warn(
/home/ahmad/envs/RL/lib/python3.10/site-packages/sklearn/base.py:413: UserWarning: X has feature names, but SVR was fitted without feature names
  warnings.warn(
/home/ahmad/envs/RL/lib/python3.10/site-packages/sklearn/base.py:413: UserWarning: X has feature names, but SVR was fitted without feature names
  warnings.warn(
/home/ahmad/envs/RL/lib/python3.10/site-packages/sklearn/base.py:413: UserWarning: X has feature names, but SVR was fitted without feature names
  warnings.warn(
/home/ahmad/envs/RL/lib/python3.10/site-packages/sklearn/base.py:413: UserWarning: X has feature names, but SVR was fitted without feature names
  warnings.warn(
/home/ahmad/envs/RL/lib/pyth

/home/ahmad/envs/RL/lib/python3.10/site-packages/sklearn/base.py:413: UserWarning: X has feature names, but SVR was fitted without feature names
  warnings.warn(
/home/ahmad/envs/RL/lib/python3.10/site-packages/sklearn/base.py:413: UserWarning: X has feature names, but SVR was fitted without feature names
  warnings.warn(
/home/ahmad/envs/RL/lib/python3.10/site-packages/sklearn/base.py:413: UserWarning: X has feature names, but SVR was fitted without feature names
  warnings.warn(
/home/ahmad/envs/RL/lib/python3.10/site-packages/sklearn/base.py:413: UserWarning: X has feature names, but SVR was fitted without feature names
  warnings.warn(
/home/ahmad/envs/RL/lib/python3.10/site-packages/sklearn/base.py:413: UserWarning: X has feature names, but SVR was fitted without feature names
  warnings.warn(
/home/ahmad/envs/RL/lib/python3.10/site-packages/sklearn/base.py:413: UserWarning: X has feature names, but SVR was fitted without feature names
  warnings.warn(
/home/ahmad/envs/RL/lib/pyth

In [75]:
print(np.mean(pehe_list))

1.0601910174705926
